In [ ]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline


In [ ]:
imgSize = (1280, 720)

calChessboardPattern = (4, 4)
calImgsPath = glob.glob("./ccalib-test-images-3/*.jpg")

objPoints = []
imgPoints = []

objPt = np.zeros((calChessboardPattern[0] * calChessboardPattern[1], 3), np.float32)
objPt[:, :2] = np.mgrid[0:calChessboardPattern[0], 0:calChessboardPattern[1]].T.reshape(-1, 2)

fig = plt.figure(figsize=(16, 14))
fig.suptitle("Part of sample chessboard images for camera calibration", fontsize=32)
columns = 3
rows = 6
i = 1

for image in calImgsPath:
    calImg = cv2.imread(image)
    ret, corners = cv2.findChessboardCorners(calImg, calChessboardPattern)
    
    if ret:
        imgPoints.append(corners)
        objPoints.append(objPt)
        cv2.drawChessboardCorners(calImg, calChessboardPattern, corners, ret)
        if i <= rows * columns:
            sub_i = fig.add_subplot(rows, columns, i)
            sub_i.set_title("cos")
            plt.imshow(calImg)
            i += 1
        else:
            break
plt.show()

In [ ]:
retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objPoints, imgPoints, imgSize, None, None)

In [ ]:
print("Camera matrix: \n", cameraMatrix)
print("\nDistortion coefficients: \n", distCoeffs)

In [ ]:
optimAlpha = 1
optimCameraMatrix, cameraROI = cv2.getOptimalNewCameraMatrix(cameraMatrix, distCoeffs, imgSize, optimAlpha)
print("Optimal camera matrix: \n", optimCameraMatrix)
print("\nRegion of interest:\n", cameraROI)

In [ ]:
def undistortImage(img):
    imgUndistRaw = imgUndist = cv2.undistort(img, cameraMatrix, distCoeffs, None, optimCameraMatrix)
    x, y, w, h = cameraROI
    x, y, w, h = int(x), int(y), int(w), int(h)
    imgUndistCrop = imgUndist[y:y + h, x:x + w]
    imgUndistScaled = cv2.resize(imgUndistCrop, imgSize)
    return imgUndistScaled, imgUndistCrop, imgUndistRaw

In [ ]:
testImg = cv2.imread("./samples/double_chessboard_1.jpg")
testUndistScaled, testUndistCrop, testUndistRaw = undistortImage(testImg)
fig = plt.figure(figsize=(16, 8))

sub1 = fig.add_subplot(2, 2, 1)
sub1.set_title("Original image")
plt.imshow(testImg)

sub2 = fig.add_subplot(2, 2, 2)
sub2.set_title("Raw undistorted image")
plt.imshow(testUndistRaw)

sub3 = fig.add_subplot(2, 2, 3)
sub3.set_title("Undistorted cropped to ROI image")
plt.imshow(testUndistCrop)
           
sub4 = fig.add_subplot(2, 2, 4)
sub4.set_title("Undistorted scaled image")
plt.imshow(testUndistScaled)
cv2.imwrite("./samples/double_chessboard_1_result.jpg", testUndistScaled)

In [ ]:
sampleRoadImgs = glob.glob("./samples/road_samples_1/*.png")

fig = plt.figure(figsize=(16, 16))
fig.suptitle("Part of sample chessboard images for camera calibration", fontsize=32)
columns = 2
rows = 4
i = 1

for image in sampleRoadImgs:
    if i <= columns * rows:
        img = cv2.imread(image)
        sub_i = fig.add_subplot(rows, columns, i)
        plt.imshow(img)
        i += 1
        testUndistScaled, testUndistCrop, testUndistRaw = undistortImage(img)
        sub_i = fig.add_subplot(rows, columns, i)
        plt.imshow(testUndistScaled)
        i += 1
    else:
        break
plt.show()